In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('pax_all_agreements_data.csv')
data.shape

(1789, 266)

In [57]:
def nas(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0:'Valores_perdidos',1 : 'Valores_Perd_Porc'})
        mz_table['Valores_Perd_Porc'] = mz_table['Valores_Perd_Porc'] # + mz_table['Zero Values'] 
        mz_table['Tipo_dato'] = df.dtypes
        mz_table = mz_table[mz_table.iloc[:,1] != 0].sort_values('Valores_Perd_Porc', ascending=False).round(1)
        print ("El data frame tiene " + str(df.shape[1]) + " columnas y " + str(df.shape[0]) + " filas.\n"      
            "Hay " + str(mz_table.shape[0]) +
              " columnas que tienen valores perdidos.")
        return mz_table
    
def cleanText(text):
    text = text.normalize('NFKD').encode('ascii', errors='ignore').decode('utf-8')
    return text

def createRange(x):
    range_year = ""
    if x >= 1990 & x <= 1997:
        range_year = "90 - 97"
    elif x > 1997 & x <= 2004:
        range_year = "97 - 04"
    elif x > 2004 & x <= 2011:
        range_year = "04 - 11"
    else:
        range_year = "11 - 18"
    
    return range_year

In [4]:
data.describe()

,PP,AgtId,Lgt,N_characters,Loc1GWNO,Loc2GWNO,UcdpAgr,PamAgr,CowWar,GCh,...,TjRep,TjRSym,TjRMa,TjNR,ImUN,ImOth,ImRef,ImPK,ImE,ImSrc
count,1782.000000,1789.000000,1789.000000,1789.000000,1769.000000,232.000000,2.970000e+02,33.000000,1111.000000,1789.000000,...,1789.000000,1789.000000,1789.000000,1789.000000,1789.000000,1789.000000,1789.000000,1789.000000,1789.000000,1789.000000
mean,89.603255,1106.604807,6.359978,11959.409726,528.292256,439.159483,2.903701e+10,17.272727,760.739874,0.313583,...,0.174958,0.010620,0.101174,0.295137,0.199553,0.241476,0.010061,0.294019,0.389603,0.999441
std,47.323516,635.813541,17.493402,37567.806638,227.494776,197.221974,3.602522e+11,10.022702,253.699787,0.773252,...,0.550695,0.102536,0.301643,0.603943,0.399776,0.428098,0.099829,0.455728,0.487797,0.023643
min,1.000000,1.000000,1.000000,0.000000,41.000000,2.000000,2.400000e+01,1.000000,139.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,547.000000,1.000000,2389.000000,359.000000,345.000000,1.107000e+03,9.000000,817.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,94.000000,1089.000000,2.000000,4362.000000,540.000000,371.000000,1.334000e+03,17.000000,857.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,120.000000,1671.000000,5.000000,9443.000000,678.000000,625.000000,1.523000e+03,26.000000,905.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,203.000000,2231.000000,323.000000,908459.000000,940.000000,770.000000,5.171991e+12,34.000000,941.000000,3.000000,...,3.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
data.head()

,Con,Contp,PP,PPName,Reg,AgtId,Agt,Dat,Status,Lgt,...,TjRep,TjRSym,TjRMa,TjNR,ImUN,ImOth,ImRef,ImPK,ImE,ImSrc
0,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,1739,Agreement between the Islamic Republic of Afgh...,2016-09-22,Multiparty signed/agreed,4,...,0,0,0,0,0,0,0,0,1,1
1,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,1923,Agreement between the two campaign teams regar...,2014-09-21,Multiparty signed/agreed,4,...,0,0,0,0,1,1,0,0,0,1
2,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,864,Tokyo Declaration Partnership for Self-Relianc...,2012-07-08,Multiparty signed/agreed,14,...,0,0,0,1,0,0,0,0,1,1
3,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,848,Conclusions of the Conference on Afghanistan a...,2011-12-05,Multiparty signed/agreed,7,...,0,0,0,1,1,0,0,0,0,1
4,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,849,Istanbul Process on Regional Security and Coop...,2011-11-02,Multiparty signed/agreed,10,...,0,0,0,1,0,0,0,0,0,1


### Selección de  Variables

In [27]:
data = data[['Con','Contp','PPName','Reg','Agt','Status','Lgt', 'N_characters','Agtp',
     'Stage','StageSub','Dat','GCh','GDis','GAge','GMig','GRa','GRe','GInd','GOth','GRef','GSoc',
     'GeWom','GeMe','GeMeNu','GeMeOth','GeSo','GeLgbti','GeFa']]

In [62]:
data['Dat'][3][2]

#data['year'] = data['Dat'].apply(createRange)
#data['year']

#  createRange(x)

'1'

### Tratamiento Valores Perdidos

In [54]:
nas(data)
data.to_csv('datos_pec4.csv', sep=',')

El data frame tiene 29 columnas y 1789 filas.
Hay 0 columnas que tienen valores perdidos.


In [42]:
data.fillna("Sin Informacion", inplace=True)

## Conteo de diferencias

### Cantidad de tratados por pais

### Tipo de tratados por pais

In [7]:
data.Contp.value_counts()

Government/territory    813
Government              662
Inter-group             206
Territory               107
Other                     1
Name: Contp, dtype: int64

### Procesos por nombre 

In [53]:
#pd.DataFrame(data.PPName.value_counts())

### Procesos por Region 

In [9]:
data.Reg.value_counts()


Africa (excl MENA)              529
Europe and Eurasia              407
Asia and Pacific                376
Middle East and North Africa    248
Americas                        195
Cross-regional                   34
Name: Reg, dtype: int64

### Procesos por Status 

In [10]:
data.Status.value_counts()

Multiparty signed/agreed            1652
Unilateral document                  120
Status unclear                        10
Agreement with subsequent status       7
Name: Status, dtype: int64

### Procesos por Agreement/Conflict 

In [11]:
data.Agtp.value_counts().sum()

1789

### Procesos por StageSub

In [12]:
data.StageSub.value_counts()

MultIss     238
Ceas        200
ImpMod      187
Iss         182
Proc        168
Rel         125
Prin        114
ExtSub      103
Conf         87
PreMix       86
FrAg         85
CeaMix       44
PreOth       43
FrCons       19
Repre        18
ExtPar       15
FrparOth     13
Reimp        13
ImpOth       12
Resub         7
Reoth         1
Name: StageSub, dtype: int64

### Procesos por Stage

In [13]:
data.Stage.value_counts()

Pre        505
SubPar     435
Cea        369
Imp        325
SubComp    106
Ren         40
Oth          9
Name: Stage, dtype: int64

In [14]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 300

In [15]:
data.head(n=20)

,Con,Contp,PP,PPName,Reg,AgtId,Agt,Dat,Status,Lgt,N_characters,Agtp,Stage,StageSub,Part,ThrdPart,OthAgr,Loc1ISO,Loc2ISO,Loc1GWNO,Loc2GWNO,UcdpCon,UcdpAgr,PamAgr,CowWar,GCh,GChRhet,GChAntid,GChSubs,GChOth,GDis,GDisRhet,GDisAntid,GDisSubs,GDisOth,GAge,GAgeRhet,GAgeAntid,GAgeSubs,GAgeOth,GMig,GMigRhet,GMigAntid,GMigSubs,GMigOth,GRa,GRaRhet,GRaAntid,GRaSubs,GRaOth,GRe,GReRhet,GReAntid,GReSubs,GReOth,GInd,GIndRhet,GIndAntid,GIndSubs,GIndOth,GOth,GOthRhet,GOthAntid,GOthSubs,GOthOth,GRef,GRefRhet,GRefSubs,GRefOth,GSoc,GSocRhet,GSocAntid,GSocSubs,GSocOth,GeWom,GeMe,GeMeNu,GeMeOth,GeSo,GeLgbti,GeLgbtiPos,GeLgbtiNeg,GeFa,StDef,StGen,StCon,StSd,StRef,StSym,StInd,StUni,StBor,StXbor,Pol,PolGen,PolNewInd,PolNewTemp,ConRen,Cons,Ele,ElecComm,PolPar,PolParTrans,PolParOth,Civso,Tral,Pubad,Polps,PpsSt,PpsSub,PpsEx,PpsOro,PpsOthPr,PpsVet,PpsAut,PpsInt,PpsOth,Terps,TpsSub,TpsLoc,TpsAut,TpsOth,Eps,EpsRes,EpsFis,EpsOth,Mps,MpsMe,MpsJt,MpsPro,MpsOth,HrGen,EqGen,HrDem,Prot,ProtCiv,ProtGrp,ProtLgl,ProtOth,HrFra,HrfSp,HrfBor,HrfTinc,HrfOth,HrCp,CprLife,CprTort,CprEq,CprSlav,CprLib,CprDet,CprFmov,CprFspe,CprFass,CprTria,CprPriv,CprVote,CprReli,CprOth,HrSec,SerProp,SerWork,SerHeal,SerEdu,SerStdl,SerShel,SerSs,SerCult,SerOth,HrNi,HrNiMe,HrNiNe,HrNiOth,HrIi,HrIiMon,HrIiBod,HrIiOth,HrMob,HrDet,Med,MedGov,MedSubs,MedLog,MedOth,HrCit,CitGen,CitRights,CitDef,CitOth,JusCr,JusCrSp,JusCrSys,JusCrPow,JusEm,JusJu,JusPri,JusTra,Dev,DevSoc,DevHum,DevInfra,NEC,NatRes,IntFu,Bus,Tax,TaxPo,TaxRef,TaxOth,Ban,CenBan,BanPers,BanInt,BanXb,LaRef,LaRefMan,LaRefRet,LaRefOth,LaNom,LaCH,LaCHTa,LaCHIt,LaCHPro,LaCHOth,LaEn,Wat,SsrGua,Ce,CeProv,CeGen,SsrPol,SsrArm,SsrDdr,DdrDemil,DdrProg,SsrInt,SsrPsf,SsrFf,Cor,SsrCrOcr,SsrDrugs,Terr,TjGen,TjAm,TjAmPro,TjSan,TjPower,TjAmBan,TjCou,TjJaNc,TjJaIc,TjMech,TjPrire,TjVet,TjVic,TjMis,TjRep,TjRSym,TjRMa,TjNR,ImUN,ImOth,ImRef,ImPK,ImE,ImSrc
0,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,1739,Agreement between the Islamic Republic of Afgh...,2016-09-22,Multiparty signed/agreed,4,10203,Intra,SubPar,MultIss,"President of the IRA Mohammad Ashraf Ghani, ch...",NaN,NaN,AFG,NaN,700.0,NaN,333,1488.0,NaN,851.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,1,1
1,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,1923,Agreement between the two campaign teams regar...,2014-09-21,Multiparty signed/agreed,4,11110,Intra,Imp,ExtSub,Dr. Mohammad Ashraf Ghani Ahmadzai \r\nDr. Abd...,The foregoing signatures were witnessed by:\r\...,"Page 3, D. Creation of the position of leader ...",AFG,NaN,700.0,NaN,333,NaN,NaN,225.0,2,1,0,1,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,2,3,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
2,Afghanistan,Government,2.0,Afghanistan: 2000s Post-intervention process,Europe and Eurasia,864,Tokyo Declaration Partnership for Self-Relianc...,2012-07-08,Multiparty signed/agreed,14,39746,InterIntra,Imp,ExtSub,"Not signed, agreement mentions the following p...","Ambassador Kazuo Kodama, Chargé d’affaires ad ...","Page 2, Preamble, 1\r\n...\r\nThese undertakin...",AFG,NaN,700.0,NaN,333,NaN,NaN,225.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Agrupaciones de Interes

In [16]:
data.groupby(['Reg']).count()

,Con,Contp,PP,PPName,AgtId,Agt,Dat,Status,Lgt,N_characters,Agtp,Stage,StageSub,Part,ThrdPart,OthAgr,Loc1ISO,Loc2ISO,Loc1GWNO,Loc2GWNO,UcdpCon,UcdpAgr,PamAgr,CowWar,GCh,GChRhet,GChAntid,GChSubs,GChOth,GDis,GDisRhet,GDisAntid,GDisSubs,GDisOth,GAge,GAgeRhet,GAgeAntid,GAgeSubs,GAgeOth,GMig,GMigRhet,GMigAntid,GMigSubs,GMigOth,GRa,GRaRhet,GRaAntid,GRaSubs,GRaOth,GRe,GReRhet,GReAntid,GReSubs,GReOth,GInd,GIndRhet,GIndAntid,GIndSubs,GIndOth,GOth,GOthRhet,GOthAntid,GOthSubs,GOthOth,GRef,GRefRhet,GRefSubs,GRefOth,GSoc,GSocRhet,GSocAntid,GSocSubs,GSocOth,GeWom,GeMe,GeMeNu,GeMeOth,GeSo,GeLgbti,GeLgbtiPos,GeLgbtiNeg,GeFa,StDef,StGen,StCon,StSd,StRef,StSym,StInd,StUni,StBor,StXbor,Pol,PolGen,PolNewInd,PolNewTemp,ConRen,Cons,Ele,ElecComm,PolPar,PolParTrans,PolParOth,Civso,Tral,Pubad,Polps,PpsSt,PpsSub,PpsEx,PpsOro,PpsOthPr,PpsVet,PpsAut,PpsInt,PpsOth,Terps,TpsSub,TpsLoc,TpsAut,TpsOth,Eps,EpsRes,EpsFis,EpsOth,Mps,MpsMe,MpsJt,MpsPro,MpsOth,HrGen,EqGen,HrDem,Prot,ProtCiv,ProtGrp,ProtLgl,ProtOth,HrFra,HrfSp,HrfBor,HrfTinc,HrfOth,HrCp,CprLife,CprTort,CprEq,CprSlav,CprLib,CprDet,CprFmov,CprFspe,CprFass,CprTria,CprPriv,CprVote,CprReli,CprOth,HrSec,SerProp,SerWork,SerHeal,SerEdu,SerStdl,SerShel,SerSs,SerCult,SerOth,HrNi,HrNiMe,HrNiNe,HrNiOth,HrIi,HrIiMon,HrIiBod,HrIiOth,HrMob,HrDet,Med,MedGov,MedSubs,MedLog,MedOth,HrCit,CitGen,CitRights,CitDef,CitOth,JusCr,JusCrSp,JusCrSys,JusCrPow,JusEm,JusJu,JusPri,JusTra,Dev,DevSoc,DevHum,DevInfra,NEC,NatRes,IntFu,Bus,Tax,TaxPo,TaxRef,TaxOth,Ban,CenBan,BanPers,BanInt,BanXb,LaRef,LaRefMan,LaRefRet,LaRefOth,LaNom,LaCH,LaCHTa,LaCHIt,LaCHPro,LaCHOth,LaEn,Wat,SsrGua,Ce,CeProv,CeGen,SsrPol,SsrArm,SsrDdr,DdrDemil,DdrProg,SsrInt,SsrPsf,SsrFf,Cor,SsrCrOcr,SsrDrugs,Terr,TjGen,TjAm,TjAmPro,TjSan,TjPower,TjAmBan,TjCou,TjJaNc,TjJaIc,TjMech,TjPrire,TjVet,TjVic,TjMis,TjRep,TjRSym,TjRMa,TjNR,ImUN,ImOth,ImRef,ImPK,ImE,ImSrc
Reg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Africa (excl MENA),529,529,528,528,529,529,529,529,529,529,529,529,518,523,360,327,524,73,524,73,441,151,18,282,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529,529
Americas,195,195,189,189,195,195,195,195,195,195,195,195,191,192,84,50,195,16,195,16,193,58,2,183,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,195,1

In [17]:
print("Variables")
var = ""
i = 1
for c in data.columns:
    var += c + ", "
    i += 1
print(var)
print(i)

Variables
Con, Contp, PP, PPName, Reg, AgtId, Agt, Dat, Status, Lgt, N_characters, Agtp, Stage, StageSub, Part, ThrdPart, OthAgr, Loc1ISO, Loc2ISO, Loc1GWNO, Loc2GWNO, UcdpCon, UcdpAgr, PamAgr, CowWar, GCh, GChRhet, GChAntid, GChSubs, GChOth, GDis, GDisRhet, GDisAntid, GDisSubs, GDisOth, GAge, GAgeRhet, GAgeAntid, GAgeSubs, GAgeOth, GMig, GMigRhet, GMigAntid, GMigSubs, GMigOth, GRa, GRaRhet, GRaAntid, GRaSubs, GRaOth, GRe, GReRhet, GReAntid, GReSubs, GReOth, GInd, GIndRhet, GIndAntid, GIndSubs, GIndOth, GOth, GOthRhet, GOthAntid, GOthSubs, GOthOth, GRef, GRefRhet, GRefSubs, GRefOth, GSoc, GSocRhet, GSocAntid, GSocSubs, GSocOth, GeWom, GeMe, GeMeNu, GeMeOth, GeSo, GeLgbti, GeLgbtiPos, GeLgbtiNeg, GeFa, StDef, StGen, StCon, StSd, StRef, StSym, StInd, StUni, StBor, StXbor, Pol, PolGen, PolNewInd, PolNewTemp, ConRen, Cons, Ele, ElecComm, PolPar, PolParTrans, PolParOth, Civso, Tral, Pubad, Polps, PpsSt, PpsSub, PpsEx, PpsOro, PpsOthPr, PpsVet, PpsAut, PpsInt, PpsOth, Terps, TpsSub, TpsLoc, 

## Variables con Valores Perdidos

In [18]:
nas(data)


El data frame tiene 266 columnas y 1789 filas.
Hay 14 columnas que tienen valores perdidos.


,Valores_perdidos,Valores_Perd_Porc,Tipo_dato
PamAgr,1756,98.2,float64
Loc2ISO,1557,87.0,object
Loc2GWNO,1557,87.0,float64
UcdpAgr,1492,83.4,float64
ThrdPart,869,48.6,object
OthAgr,852,47.6,object
CowWar,678,37.9,float64
UcdpCon,146,8.2,object
Loc1ISO,44,2.5,object
StageSub,29,1.6,object


### PamAgr
Kroc Institute’s Peace Accords Matrix agreement ID. It is possible that this field is empty –
this happens in cases of agreements not part of the PAM data. The PAM focuses on comprehensive agreements only, so the bulk of PA-X documents do not have the PAM agreement
code. 

### Loc2Iso
Only used in cases of interstate conflict, indicating one of the primary locations of intrastate
conflict, and returning the ISO country designation. The two main locations are assigned
randomly to location 1 or location 2.

### Loc2GWNO
Only used in cases of interstate conflict, indicating one of the primary locations of intrastate
conflict, and returning the Gleditsch-Ward country code. The two main locations are assigned randomly to location 1 or location 2. 